## 1.

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import requests
import json
import os
import datetime
import time

In [10]:
def pass_web(date, ssn, web): 
    
    ### the 'team' parameter takes in an nba team's three letter abbreviation. 'GSW' for the golden state warriors, for example
    ### the 'ssn' parameter takes in an nba season (2013-14 at the earliest). '2015-16' for example
    ### the 'web' parameter takes in 'AST' or 'pass' (or other variations of the two). 
    ### the 'web' parameter exists because you can create networks in which the lines represent either assists or passes -- the parameter is used to tell which one of the two you'd like to do
    ### example of usage: pass_web('GSW','2016-17','AST')
    
#     if (web in ['ast','AST','assist','assists','assisting','ASSIST','ASSISTS']):
#         web = 'assist'
#     elif (web in ['pass','passes','passing','PASS','PASSES']):
#         web = 'pass'
#     else:
#         return print('Error: Third parameter requires "assist" or "pass" input')
    
    # set headers  
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}
    
    url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
    json = requests.get(url, headers=headers).json()
    data = json['resultSets'][0]['rowSet']
    columns = json['resultSets'][0]['headers']
    tms = pd.DataFrame.from_records(data, columns=columns)
    teamName_dict = dict(zip(tms['TEAM_ID'], tms['TEAM_ABBREVIATION']))
    teamId_dict = dict(zip(tms['TEAM_ID'], [0 for i in range(len(tms['TEAM_ID']))]))
    win_dict = dict(zip(tms['TEAM_ID'], tms['W']))
    schedule_list = []
    while 0 in teamId_dict.values():
        for key in teamId_dict:
            if teamId_dict[key] == 0:
                url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=' + str(key) + '&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
                json = requests.get(url, headers=headers).json()
                data = json['resultSets'][0]['rowSet']
                columns = json['resultSets'][0]['headers']
                oppo = pd.DataFrame.from_records(data, columns=columns)['TEAM_ID'][0]
                teamId_dict[key] = 1
                teamId_dict[oppo] = 1
                schedule_list.append({key: win_dict[key], oppo: win_dict[oppo]})
                break
    
    ### using the scraped team_id, find all players who accumulated at least 10 assists with that team in the inputted season
    if not os.path.exists('./data'):
        os.mkdir('./data')
    date_name = date.replace('/', '-')
    if not os.path.exists(os.path.join('data', date_name)):
        os.mkdir(os.path.join('data', date_name))
    
    for team_id in schedule_list:
        time.sleep(0.5)
        team_ids = list(team_id.keys())
        url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=' + str(team_ids[0]) + '&TwoWay=0&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        team_df = pd.DataFrame.from_records(data, columns=columns)
        team_df.to_csv(os.path.join('data', date_name, teamName_dict[team_ids[0]] + '.csv'))
        time.sleep(0.5)
        url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=' + str(team_ids[1]) + '&TwoWay=0&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        team_df = pd.DataFrame.from_records(data, columns=columns)
        team_df.to_csv(os.path.join('data', date_name, teamName_dict[team_ids[1]] + '.csv'))
        
    return schedule_list

In [11]:
# test
# list_1 = pass_web('12/25/2016','2016-17','AST')

In [12]:
schedule_list = dict()
data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start='10/18/2022', end='04/09/2023')]
for data in data_list:
    print(data)
    list_schedule = pass_web(data,'2022-23','AST')
    schedule_list[data] = list_schedule

10/18/2022
10/19/2022
10/20/2022
10/21/2022
10/22/2022
10/23/2022
10/24/2022
10/25/2022
10/26/2022
10/27/2022
10/28/2022
10/29/2022
10/30/2022
10/31/2022
11/01/2022
11/02/2022
11/03/2022
11/04/2022
11/05/2022
11/06/2022
11/07/2022
11/08/2022
11/09/2022
11/10/2022
11/11/2022
11/12/2022
11/13/2022
11/14/2022
11/15/2022
11/16/2022
11/17/2022
11/18/2022
11/19/2022
11/20/2022
11/21/2022
11/22/2022
11/23/2022
11/24/2022
11/25/2022
11/26/2022
11/27/2022
11/28/2022
11/29/2022
11/30/2022
12/01/2022
12/02/2022
12/03/2022
12/04/2022
12/05/2022
12/06/2022
12/07/2022
12/08/2022
12/09/2022
12/10/2022
12/11/2022
12/12/2022
12/13/2022
12/14/2022
12/15/2022
12/16/2022
12/17/2022
12/18/2022
12/19/2022
12/20/2022
12/21/2022
12/22/2022
12/23/2022
12/24/2022
12/25/2022
12/26/2022
12/27/2022
12/28/2022
12/29/2022
12/30/2022
12/31/2022
01/01/2023
01/02/2023
01/03/2023
01/04/2023
01/05/2023
01/06/2023
01/07/2023
01/08/2023
01/09/2023
01/10/2023
01/11/2023
01/12/2023
01/13/2023
01/14/2023
01/15/2023
01/16/2023

In [13]:
schedule_list

{'10/18/2022': [{1610612738: 1, 1610612755: 0},
  {1610612744: 1, 1610612747: 0}],
 '10/19/2022': [{1610612737: 1, 1610612745: 0},
  {1610612751: 0, 1610612740: 1},
  {1610612766: 1, 1610612759: 0},
  {1610612741: 1, 1610612748: 0},
  {1610612739: 0, 1610612761: 1},
  {1610612742: 0, 1610612756: 1},
  {1610612743: 0, 1610612762: 1},
  {1610612765: 1, 1610612753: 0},
  {1610612754: 0, 1610612764: 1},
  {1610612763: 1, 1610612752: 0},
  {1610612750: 1, 1610612760: 0},
  {1610612757: 1, 1610612758: 0}],
 '10/20/2022': [{1610612746: 1, 1610612747: 0},
  {1610612749: 1, 1610612755: 0}],
 '10/21/2022': [{1610612737: 1, 1610612753: 0},
  {1610612738: 1, 1610612748: 0},
  {1610612751: 1, 1610612761: 0},
  {1610612766: 0, 1610612740: 1},
  {1610612741: 0, 1610612764: 1},
  {1610612743: 1, 1610612744: 0},
  {1610612765: 0, 1610612752: 1},
  {1610612745: 0, 1610612763: 1},
  {1610612754: 0, 1610612759: 1},
  {1610612750: 0, 1610612762: 1},
  {1610612756: 0, 1610612757: 1}],
 '10/22/2022': [{16106

In [14]:
import pickle
pickle.dump(schedule_list, open( "schedule4.p", "wb" ))

## 2. 

In [2]:
import pickle
schedule_list = pickle.load(open( "schedule4.p", "rb" ))

In [3]:
schedule_list

{'10/18/2022': [{1610612738: 1, 1610612755: 0},
  {1610612744: 1, 1610612747: 0}],
 '10/19/2022': [{1610612737: 1, 1610612745: 0},
  {1610612751: 0, 1610612740: 1},
  {1610612766: 1, 1610612759: 0},
  {1610612741: 1, 1610612748: 0},
  {1610612739: 0, 1610612761: 1},
  {1610612742: 0, 1610612756: 1},
  {1610612743: 0, 1610612762: 1},
  {1610612765: 1, 1610612753: 0},
  {1610612754: 0, 1610612764: 1},
  {1610612763: 1, 1610612752: 0},
  {1610612750: 1, 1610612760: 0},
  {1610612757: 1, 1610612758: 0}],
 '10/20/2022': [{1610612746: 1, 1610612747: 0},
  {1610612749: 1, 1610612755: 0}],
 '10/21/2022': [{1610612737: 1, 1610612753: 0},
  {1610612738: 1, 1610612748: 0},
  {1610612751: 1, 1610612761: 0},
  {1610612766: 0, 1610612740: 1},
  {1610612741: 0, 1610612764: 1},
  {1610612743: 1, 1610612744: 0},
  {1610612765: 0, 1610612752: 1},
  {1610612745: 0, 1610612763: 1},
  {1610612754: 0, 1610612759: 1},
  {1610612750: 0, 1610612762: 1},
  {1610612756: 0, 1610612757: 1}],
 '10/22/2022': [{16106

In [4]:
moral_dict = dict()
for key in schedule_list:
    match_list = schedule_list[key]
    
    for match_dict in match_list:
        for tid in match_dict:
            if tid in moral_dict:
                moral_dict[tid][key] = match_dict[tid]
            else:
                moral_dict[tid] = {key: match_dict[tid]}

## passing

In [2]:
def pass_data(date, ssn): 
    print(date)
    error_list = []
    ### the 'team' parameter takes in an nba team's three letter abbreviation. 'GSW' for the golden state warriors, for example
    ### the 'ssn' parameter takes in an nba season (2013-14 at the earliest). '2015-16' for example
    ### the 'web' parameter takes in 'AST' or 'pass' (or other variations of the two). 
    ### the 'web' parameter exists because you can create networks in which the lines represent either assists or passes -- the parameter is used to tell which one of the two you'd like to do
    ### example of usage: pass_web('GSW','2016-17','AST')
    
    web = 'pass'
    
    # set headers  
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}
    
    url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
    json = requests.get(url, headers=headers).json()
    data = json['resultSets'][0]['rowSet']
    columns = json['resultSets'][0]['headers']
    tms = pd.DataFrame.from_records(data, columns=columns)
    teamName_dict = dict(zip(tms['TEAM_ID'], tms['TEAM_ABBREVIATION']))
    teams = tms['TEAM_ID']
    
    if not os.path.exists('./pass'):
        os.mkdir('./pass')
    date_name = date.replace('/', '-')
    if not os.path.exists(os.path.join('pass', date_name)):
        os.mkdir(os.path.join('pass', date_name))
    
#     print(teams)
    
    for t in teams:
        time.sleep(1)
        url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=' + str(t) + '&TwoWay=0&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        df = pd.DataFrame.from_records(data, columns=columns)
        players = df.PLAYER_ID.unique()
        df_list = []
        for x in players:
#             print(x)
            time.sleep(1)
            url = 'https://stats.nba.com/stats/playerdashptpass?DateFrom=' + str(date) + '&DateTo=' + str(date) + '&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerID=' + str(x) + '&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision='
            try:
                json = requests.get(url, headers=headers).json()
            except ValueError:  # This is the correct syntax
                error_list.append(t)
                print("===============", t, "=====================")
                pass
            data = json['resultSets'][0]['rowSet']
            columns = json['resultSets'][0]['headers']
            df_list.append(pd.DataFrame.from_records(data, columns=columns))
        time.sleep(0.5)
        df = pd.concat(df_list)
        if len(df['TEAM_ABBREVIATION']) != 0:
            name = list(df['TEAM_ABBREVIATION'][0])[0]
        df = df[['TEAM_ID','PLAYER_NAME_LAST_FIRST','PASS_TO','PASS','AST']]
        df.columns = ['tm','passer','receiver','passes','ast']
        df.to_csv(os.path.join('pass', date_name, name + '.csv'))
        
    return teamName_dict, list(set(error_list))

In [3]:
data_list = [d.strftime('%m/%d/%Y') for d in pd.date_range(start='01/26/2023', end='04/09/2023')]
teamN_dict = dict()
error_dict = dict()
for data in data_list:
    team_dict, err = pass_data(data, '2022-23')
    error_dict[data] = err
    for k in team_dict:
        if k not in teamN_dict:
            teamN_dict[k] = team_dict[k]

01/26/2023
01/27/2023
01/28/2023
01/29/2023
01/30/2023
01/31/2023
02/01/2023
02/02/2023
02/03/2023
02/04/2023
02/05/2023
02/06/2023
02/07/2023
02/08/2023
02/09/2023
02/10/2023
02/11/2023
02/12/2023
02/13/2023
02/14/2023
02/15/2023
02/16/2023
02/17/2023
02/18/2023
02/19/2023
02/20/2023
02/21/2023
02/22/2023
02/23/2023
02/24/2023
02/25/2023
02/26/2023
02/27/2023
02/28/2023
03/01/2023
03/02/2023
03/03/2023
03/04/2023
03/05/2023
03/06/2023
03/07/2023
03/08/2023
03/09/2023
03/10/2023
03/11/2023
03/12/2023
03/13/2023
03/14/2023
03/15/2023
03/16/2023
03/17/2023
03/18/2023
03/19/2023
03/20/2023
03/21/2023
03/22/2023
03/23/2023
03/24/2023
03/25/2023
03/26/2023
03/27/2023
03/28/2023
03/29/2023
03/30/2023
03/31/2023
04/01/2023
04/02/2023
04/03/2023
04/04/2023
04/05/2023
04/06/2023
04/07/2023
04/08/2023
04/09/2023


In [5]:
error_dict

{'12/08/2021': [],
 '12/09/2021': [],
 '12/10/2021': [],
 '12/11/2021': [],
 '12/12/2021': [],
 '12/13/2021': [],
 '12/14/2021': [],
 '12/15/2021': [],
 '12/16/2021': [],
 '12/17/2021': [],
 '12/18/2021': [],
 '12/19/2021': [],
 '12/20/2021': [],
 '12/21/2021': [],
 '12/22/2021': [],
 '12/23/2021': [1610612755],
 '12/24/2021': [],
 '12/25/2021': [],
 '12/26/2021': [],
 '12/27/2021': [],
 '12/28/2021': [],
 '12/29/2021': [],
 '12/30/2021': [],
 '12/31/2021': [],
 '01/01/2022': [],
 '01/02/2022': [],
 '01/03/2022': [],
 '01/04/2022': [],
 '01/05/2022': [],
 '01/06/2022': [],
 '01/07/2022': [],
 '01/08/2022': [],
 '01/09/2022': [],
 '01/10/2022': [],
 '01/11/2022': [],
 '01/12/2022': [],
 '01/13/2022': [],
 '01/14/2022': [],
 '01/15/2022': [],
 '01/16/2022': [],
 '01/17/2022': [],
 '01/18/2022': [],
 '01/19/2022': [],
 '01/20/2022': [],
 '01/21/2022': [],
 '01/22/2022': [],
 '01/23/2022': [],
 '01/24/2022': [],
 '01/25/2022': [],
 '01/26/2022': [],
 '01/27/2022': [],
 '01/28/2022': [],
 '

In [ ]:
import pickle
pickle.dump(error_dict, open( "error.p", "wb" ))

In [28]:
import pickle
pickle.dump(teamN_dict, open( "team.p", "wb" ))

In [27]:
len(teamN_dict.keys())

30

In [25]:
error_dict

{'01/01/2021': [],
 '01/02/2021': [],
 '01/03/2021': [],
 '01/04/2021': [],
 '01/05/2021': [],
 '01/06/2021': [],
 '01/07/2021': [],
 '01/08/2021': [],
 '01/09/2021': [],
 '01/10/2021': []}

## error

In [2]:
def pass_data_error(date, ssn, tid): 
    print(date)
    
    ### the 'team' parameter takes in an nba team's three letter abbreviation. 'GSW' for the golden state warriors, for example
    ### the 'ssn' parameter takes in an nba season (2013-14 at the earliest). '2015-16' for example
    ### the 'web' parameter takes in 'AST' or 'pass' (or other variations of the two). 
    ### the 'web' parameter exists because you can create networks in which the lines represent either assists or passes -- the parameter is used to tell which one of the two you'd like to do
    ### example of usage: pass_web('GSW','2016-17','AST')
    
    web = 'pass'
    
    # set headers  
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}
    
    url = 'https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerOrTeam=Team&PlayerPosition=&PtMeasureType=Drives&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
    json = requests.get(url, headers=headers).json()
    data = json['resultSets'][0]['rowSet']
    columns = json['resultSets'][0]['headers']
    tms = pd.DataFrame.from_records(data, columns=columns)
    teamName_dict = dict(zip(tms['TEAM_ID'], tms['TEAM_ABBREVIATION']))
    teams = tms['TEAM_ID']
    
    if not os.path.exists('./pass'):
        os.mkdir('./pass')
    date_name = date.replace('/', '-')
    if not os.path.exists(os.path.join('pass', date_name)):
        os.mkdir(os.path.join('pass', date_name))
    
    for t in [tid]:
        time.sleep(0.5)
        url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=' + str(date) + '&DateTo=' + str(date) + '&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=' + str(t) + '&TwoWay=0&VsConference=&VsDivision=&Weight='
        json = requests.get(url, headers=headers).json()
        data = json['resultSets'][0]['rowSet']
        columns = json['resultSets'][0]['headers']
        df = pd.DataFrame.from_records(data, columns=columns)
        players = df.PLAYER_ID.unique()
        df_list = []
        for x in players:
            time.sleep(1)
            url = 'https://stats.nba.com/stats/playerdashptpass?DateFrom=' + str(date) + '&DateTo=' + str(date) + '&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerID=' + str(x) + '&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision='
            try:
                json = requests.get(url, headers=headers).json()
            except requests.exceptions.RequestException as e:  # This is the correct syntax
                print(t)
            data = json['resultSets'][0]['rowSet']
            columns = json['resultSets'][0]['headers']
            df_list.append(pd.DataFrame.from_records(data, columns=columns))
        time.sleep(1)
        df = pd.concat(df_list)
        name = list(df['TEAM_ABBREVIATION'][0])[0]
        df = df[['TEAM_ID','PLAYER_NAME_LAST_FIRST','PASS_TO','PASS','AST']]
        df.columns = ['tm','passer','receiver','passes','ast']
        df.to_csv(os.path.join('pass', date_name, name + '.csv'))

In [3]:
pass_data_error('01/20/2020', '2019-20', '1610612739')

01/20/2020
